In [ ]:
# Cell 1: imports and seeding
import numpy as np
import matplotlib.pyplot as plt
from Task1.model import Dense, ReLU, Sigmoid, MSELoss, SGD, NeuralNet
np.random.seed(42)


In [ ]:
# Cell 2: generate synthetic dataset (noisy cubic function)
def make_data(n=1000, noise_std=4.0):
    X = np.random.uniform(-3, 3, size=(n, 1))
    # non-trivial function: cubic + quadratic + linear
    y = 0.5 * X[:, 0]**3 - 1.2 * X[:, 0]**2 + 0.3 * X[:, 0] + 2.0
    y = y.reshape(-1, 1)
    y = y + np.random.normal(scale=noise_std, size=y.shape)
    return X.astype(np.float64), y.astype(np.float64)

X, y = make_data(2000, noise_std=6.0)
plt.scatter(X, y, s=6); plt.title("Training data (noisy cubic)")


In [ ]:
# Cell 3: build network
layers = [
    Dense(1, 64, seed=1),
    ReLU(),
    Dense(64, 64, seed=2),
    ReLU(),
    Dense(64, 1, seed=3)
]
net = NeuralNet(layers)
loss_fn = MSELoss()
opt = SGD(lr=1e-3)


In [ ]:
# Cell 4: training loop with mini-batches
def iterate_minibatches(X, y, batch_size=64, shuffle=True):
    idx = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(idx)
    for i in range(0, X.shape[0], batch_size):
        batch_idx = idx[i:i+batch_size]
        yield X[batch_idx], y[batch_idx]

epochs = 300
batch_size = 64
loss_history = []

for ep in range(epochs):
    epoch_loss = 0.0
    count = 0
    for xb, yb in iterate_minibatches(X, y, batch_size):
        preds = net.forward(xb)
        loss = loss_fn.forward(preds, yb)
        epoch_loss += loss * xb.shape[0]
        count += xb.shape[0]
        # backward
        dloss = loss_fn.backward()  # gradient w.r.t preds
        net.backward(dloss)
        # collect params and grads and update
        opt.step(net.parameters_and_grads())
    loss_history.append(epoch_loss / count)
    if (ep+1) % 50 == 0:
        print(f"Epoch {ep+1}/{epochs} loss: {loss_history[-1]:.4f}")


In [ ]:
# Cell 5: plot loss curve
plt.plot(loss_history)
plt.yscale('log')
plt.xlabel('Epoch'); plt.ylabel('MSE loss (log scale)')
plt.title('Training loss')


In [ ]:
# Cell 6: final prediction vs ground truth
X_test = np.linspace(-3.5, 3.5, 400).reshape(-1,1)
y_true = 0.5 * X_test[:, 0]**3 - 1.2 * X_test[:, 0]**2 + 0.3 * X_test[:, 0] + 2.0
y_pred = net.forward(X_test)

plt.scatter(X, y, s=6, alpha=0.2, label='train')
plt.plot(X_test, y_true, label='ground truth', linewidth=2)
plt.plot(X_test, y_pred, label='prediction', linewidth=2)
plt.legend(); plt.title('Prediction vs ground truth')
